Test if delta NPI > 0.7 is associated with DCI

Privitera et al. BMC Neurology (2022) 22:273 https://doi.org/10.1186/s12883-022-02801-3

In [ ]:
import pandas as pd
import os

In [ ]:
path = '/Users/jk1/temp/cereblink/pupillometry/data_saving/exclude_nan_outcome_False/'

In [ ]:
# find all files like DCI_ischemia_reassembled_pupillometry_Xh_timebin.csv and load them


files = os.listdir(path)
files = [f for f in files if 'DCI_ischemia_reassembled_pupillometry' in f and f.endswith('timebin.csv')]
dfs = {}
for f in files:
    df = pd.read_csv(path + f)
    timebin = int(f.split('_')[4][:-1])
    dfs[timebin] = df

In [ ]:
# roc curve for NPI_inter_eye_delta_timebin_max
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

temp_df = df[['label', 'NPI_inter_eye_delta_timebin_median']]
temp_df = temp_df.dropna()

fpr, tpr, _ = roc_curve(temp_df.label.values, temp_df.NPI_inter_eye_delta_timebin_median.values)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from pupillometry.marker_evaluation.testing_utils import test_predictor


markers = ['NPI_inter_eye_delta_timebin_median', 'NPI_inter_eye_delta_timebin_max', 'NPI_inter_eye_delta_timebin_min']
overall_results = []

# loop through all timebins
for timebin, df in dfs.items():
    for marker in markers:
        results = test_predictor(df.label.values, (df[marker] > 0.7).astype(int).values)
        results_df = pd.DataFrame(results, index=[marker])
        results_df['timebin'] = timebin
        overall_results.append(results_df)
    
overall_results_df = pd.concat(overall_results)


In [ ]:
overall_results_df.reset_index(inplace=True)
overall_results_df.rename(columns={'index': 'marker'}, inplace=True)

In [ ]:
overall_results_df

In [ ]:
overall_results_df['threshold'] = 0.7